In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix

In [ ]:

input_dir="C:\\Users\\91773\\Desktop\\Mangoes to be cropped"
output_dir="C:\\Users\\91773\\Desktop\\Cropped Mangoes\\"

In [ ]:


count=0
for file_name in os.listdir(input_dir):
    file_path=os.path.join(input_dir, file_name)
    img=cv2.imread(file_path)
    file_name=file_name.split('_')[2].split('.')[0]
    res1=img[90:700,0:550]
    count+=1
    cnt=str(count)
    cv2.imwrite(output_dir+file_name+cnt+".png", res1)
    res2=img[710:1165,0:550]
    count+=1
    cnt=str(count)
    cv2.imwrite(output_dir+file_name+cnt+".png", res2)
    res3=img[1170:1700,0:650]
    count+=1
    cnt=str(count)
    cv2.imwrite(output_dir+file_name+cnt+".png", res3)
    res4=img[0:675,550:1400]
    count+=1
    cnt=str(count)
    cv2.imwrite(output_dir+file_name+cnt+".png", res4)
    res5=img[675:1200,550:1300]
    count+=1
    cnt=str(count)
    cv2.imwrite(output_dir+file_name+cnt+".png", res5)
    res6=img[1170:1760,630:1410]
    count+=1
    cnt=str(count)
    cv2.imwrite(output_dir+file_name+cnt+".png", res6)
    res7=img[0:600,1410:2200]
    count+=1
    cnt=str(count)
    cv2.imwrite(output_dir+file_name+cnt+".png", res7)
    res8=img[580:1300,1310:2200]
    count+=1
    cnt=str(count)
    cv2.imwrite(output_dir+file_name+cnt+".png", res8)
    res9=img[1250:1750,1400:2200]
    count+=1
    cnt=str(count)
    cv2.imwrite(output_dir+file_name+cnt+".png", res9)

In [ ]:


def split_image(image):
    res1=image[90:700,0:550]
    res2=image[710:1165,0:550]
    res3=image[1170:1700,0:650]
    res4=image[0:675,550:1400]
    res5=image[675:1200,550:1300]
    res6=image[1170:1760,630:1410]
    res7=image[0:600,1410:2200]
    res8=image[580:1300,1310:2200]
    res9=image[1250:1750,1400:2200]
    li=[]
    li.append(res1)
    li.append(res2)
    li.append(res3)
    li.append(res4)
    li.append(res5)
    li.append(res6)
    li.append(res7)
    li.append(res8)
    li.append(res9)
    return li

In [ ]:


def feature_extraction(img):
    # Calculate the histogram of the image
    li=[]
    b, g, r=cv2.split(img)
    li.append(b.mean())
    li.append(g.mean())
    li.append(r.mean())
    hist, _ = np.histogram(img, bins=256)
    
    # Normalize the histogram to have values between 0 and 1
    hist_norm = hist / (img.shape[0] * img.shape[1])
    
    # Calculate the entropy of the image
    entropy = -np.sum(hist_norm * np.log2(hist_norm + 1e-7))
    li.append(entropy)
    # Print the entropy of the image
    #print("Entropy: ", entropy)
    
    # Convert the image from BGR to RGB color space
    img1 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Convert the RGB image to HSI color space
    hsi_img = cv2.cvtColor(img1, cv2.COLOR_RGB2HSV)
    
    # Split the HSI image into its components
    hue, sat, intensity = cv2.split(hsi_img)
    
    # Calculate the mean values of the hue, saturation, and intensity channels
    hue_mean = np.mean(hue)
    sat_mean = np.mean(sat)
    intensity_mean = np.mean(intensity)
    li.append(hue_mean)
    li.append(sat_mean)
    li.append(intensity_mean)
    # Print the HSI values of the image
    #print("Hue: ", hue_mean)
    #print("Saturation: ", sat_mean)
    #print("Intensity: ", intensity_mean)
    
    # Convert the image to grayscale
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    r = img[:,:,2]
    r = cv2.GaussianBlur(r, (0,0), 7)
    
    # Binarize red channel using mean threshold
    red_avg = np.mean(r[r != 0])
    dark_spots = r.copy()
    dark_spots[dark_spots > red_avg] = 0
    dark_spots = cv2.threshold(dark_spots, 0, 255, cv2.THRESH_BINARY)[1]
    
    # Extract saturation channel from HSV and use as metric for light spots
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    s = hsv[:,:,1]
    s_avg = np.mean(s[s != 0])
    light_spots = s.copy()
    light_spots[light_spots > 0.7*s_avg] = 0
    light_spots = cv2.threshold(light_spots, 0, 255, cv2.THRESH_BINARY)[1]
    
    # Combine dark and light spots and process
    spots = cv2.bitwise_or(dark_spots, light_spots)
    mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    mask[16:-16,16:-16] = 1
    mask = cv2.dilate(mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (16,16)))
    spots = cv2.bitwise_and(spots, cv2.bitwise_not(mask))
    spots = cv2.dilate(spots, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2,2)))
    contours, hierarchy = cv2.findContours(spots, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Remove calyx and small segments
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 40:
            continue
        epsilon = 0.1 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)
        if len(approx) < 5:
            continue
        ellipse = cv2.fitEllipse(approx)
        center = ellipse[0]
        axes = ellipse[1]
        roundness = axes[0] / axes[1]
        if roundness < 0.1:
            continue
        dist = np.sqrt((500-center[0])**2 + (500-center[1])**2)
        if dist < 85 or area / np.sum(mask) < 0.03:
            cv2.drawContours(spots, [cnt], -1, 0, -1)
    
    # Compute percent blemished
    percent_blemished = (np.sum(spots != 0) / np.sum(mask))*100
    li.append(percent_blemished)
    return li

In [ ]:


img_dir_grade1="C:\\Users\\91773\\Desktop\\G1"
img_dir_grade2="C:\\Users\\91773\\Desktop\\G2"
img_dir_grade3="C:\\Users\\91773\\Desktop\\G3"

In [ ]:


labels = ["G1", "G2", "G3"]
columns=['Blue Component','Green Component','Red Component','Entropy','Hue','Saturation','Intensity','Percent Defect','Grade']
df=pd.DataFrame(columns=columns)
df

In [ ]:


for img in os.listdir(img_dir_grade1):
    file_path=os.path.join(img_dir_grade1, img)
    img=cv2.imread(file_path)
    feature=feature_extraction(img)
    feature.append("G1")
    new_df=pd.DataFrame([feature],columns=df.columns)
    new_df['Percent Defect']=new_df['Percent Defect']
    df=pd.concat([df, new_df], ignore_index=True)
df

In [ ]:


df[df['Grade']=="G1"]['Green Component'].mean(), df[df['Grade']=="G1"]['Red Component'].mean(), df[df['Grade']=="G1"]['Blue Component'].mean(), df[df['Grade']=="G1"]['Percent Defect'].mean()

In [ ]:


for img in os.listdir(img_dir_grade2):
    file_path=os.path.join(img_dir_grade2, img)
    img=cv2.imread(file_path)
    feature=feature_extraction(img)
    feature.append("G2")
    new_df=pd.DataFrame([feature],columns=df.columns)
    new_df['Percent Defect']=new_df['Percent Defect']
    df=pd.concat([df, new_df], ignore_index=True)
df[df['Grade']=='G2']

In [ ]:


df[df['Grade']=='G2']['Green Component'].mean(), df[df['Grade']=='G2']['Red Component'].mean(), df[df['Grade']=='G2']['Blue Component'].mean(),  df[df['Grade']=='G2']['Percent Defect'].mean()

In [ ]:


for img in os.listdir(img_dir_grade3):
    file_path=os.path.join(img_dir_grade3, img)
    img=cv2.imread(file_path)
    feature=feature_extraction(img)
    feature.append("G3")
    new_df=pd.DataFrame([feature],columns=df.columns)
    df=pd.concat([df, new_df], ignore_index=True)
df[df['Grade']=='G3']

In [ ]:


df[df['Grade']=='G3']['Green Component'].mean(), df[df['Grade']=='G3']['Red Component'].mean(), df[df['Grade']=='G3']['Blue Component'].mean(),  df[df['Grade']=='G3']['Percent Defect'].mean()

In [ ]:


# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], test_size=0.2, random_state=42)

# Creating the neural network
model = Sequential()
model.add(Dense(16, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))

# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the model
num_epochs=81
model.fit(X_train, pd.get_dummies(y_train), epochs=num_epochs, batch_size=10)

# Testing the model
_, accuracy = model.evaluate(X_test, pd.get_dummies(y_test))
print('Accuracy: %.2f' % (accuracy*100))

In [ ]:


y_pred=model.predict(X_test)
Y_pred = np.array([labels[idx] for idx in np.argmax(y_pred, axis=1)])
cm=confusion_matrix(y_test, Y_pred)
cm

In [ ]:


df_1=pd.DataFrame(columns=columns[:-1])
images_path="C:\\Users\\91773\\Downloads\\output_data\\"
image=cv2.imread(images_path+"2021-08-11 06_55_00.png")
images_list=split_image(image)
for i in images_list:
    feat=feature_extraction(i)
    newdf=pd.DataFrame([feat],columns=columns[:-1])
    df_1=pd.concat([df_1,newdf], ignore_index=True)
df_1

In [ ]:


def predict_():
    X_pred=df_1
    grade_predicted=model.predict(X_pred)
    predicted_labels = np.array([labels[idx] for idx in np.argmax(grade_predicted, axis=1)])
    return predicted_labels

In [ ]:


count={'day1':{"G1":0,"G2":0,"G3":0},'day2':{"G1":0,"G2":0,"G3":0},'day3':{"G1":0,"G2":0,"G3":0},'day4':{"G1":0,"G2":0,"G3":0},'day5':{"G1":0,"G2":0,"G3":0}}

In [ ]:


predictions=predict_()
for g in predictions:
    if g=="G1":
        count['day2']["G1"]+=1
    elif g=="G2":
        count['day2']["G2"]+=1
    elif g=="G3":
        count['day2']["G3"]+=1

In [ ]:


count['day1']["G1"], count['day1']["G2"], count['day1']["G3"]

In [ ]:


count['day2']["G1"], count['day2']["G2"], count['day2']["G3"]

In [ ]:


count['day3']["G1"], count['day3']["G2"], count['day3']["G3"]

In [ ]:


count['day4']["G1"], count['day4']["G2"], count['day4']["G3"]

In [ ]:


count['day5']["G1"], count['day5']["G2"], count['day5']["G3"]

In [ ]:


g1=[]
g2=[]
g3=[]
g1.append(count['day1']["G1"])
g1.append(count['day2']["G1"])
g1.append(count['day3']["G1"])
g1.append(count['day4']["G1"])
g1.append(count['day5']["G1"])
g2.append(count['day1']["G2"])
g2.append(count['day2']["G2"])
g2.append(count['day3']["G2"])
g2.append(count['day4']["G2"])
g2.append(count['day5']["G2"])
g3.append(count['day1']["G3"])
g3.append(count['day2']["G3"])
g3.append(count['day3']["G3"])
g3.append(count['day4']["G3"])
g3.append(count['day5']["G3"])
g1,g2,g3

In [ ]:


x=["Week0", "Week1", "Week2", "Week3", "Week4"]
plt.plot(x,g1,label='g1')
plt.plot(x,g2,label='g2')
plt.plot(x,g3,label='g3')
plt.title("Weekly Degradation Trend")
plt.legend()
plt.show()


# We Define Shelf Life of a batch of mango in terms of its economic value in the market. Empirically Speaking, It is defined as the point in time where the number of g3 mangoes (The ones which cannot be sold) exceed the number of g2 mangoes (The ones which can be sold at some discounted price atleast), Since the batch is then believed to have lost the market value. Thus the transport has to take place before this time period, which in our case lies just before the end of 3rd week, i.e, somewhere around 18-20 days.